In [ ]:
 !pip install tenseal

**Prelims**

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array
from tensorflow.keras.datasets import cifar10
import matplotlib.pyplot as plt
from collections import defaultdict
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Softmax, Input
from tensorflow.keras.optimizers import SGD
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import load_model,  Model
from sklearn.metrics import classification_report
import tenseal as ts
import numpy as np
import glob
import os
import cv2

Pre-Trained model

In [ ]:
# scale pixels
def prep_pixels(train, test):
    # convert from integers to floats
    train_norm = train.astype('float32')
    test_norm = test.astype('float32')
    # normalize to range 0-1
    train_norm = train_norm / 255.0
    test_norm = test_norm / 255.0
    # return normalized images
    return train_norm, test_norm

def load_dataset():
    # Load the CIFAR-10 dataset
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    # Initialize empty lists to store selected images and labels
    selected_images = []
    selected_labels = []
    test_images = []
    test_labels = []

    # Iterate through the first four labels for training data
    for label in range(4):
        # Get indices of images with the current label
        indices = np.where(y_train.squeeze() == label)[0]
        selected_indices = indices[4000:5000]  #Since we want a subset of the first 1000 images
        selected_images.extend(x_train[selected_indices])
        selected_labels.extend(y_train[selected_indices])

    # Iterate through the first four labels for test data
    for label in range(4):
        # Get indices of images with the current label
        indices = np.where(y_test.squeeze() == label)[0]
        test_images.extend(x_test[indices])
        test_labels.extend(y_test[indices])

    # Convert the lists to NumPy arrays
    selected_images = np.array(selected_images)
    selected_labels = np.array(selected_labels)
    test_images = np.array(test_images)
    test_labels = np.array(test_labels)

    # One-hot encode target values
    selected_labels = to_categorical(selected_labels, num_classes=4)
    test_labels = to_categorical(test_labels, num_classes=4)

    return selected_images, selected_labels, test_images, test_labels


def define_cifar10_model():
    inputs = Input(shape=(32, 32, 3))

    x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(inputs)
    x = Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(64, activation='relu', kernel_initializer='he_uniform')(x)

    x = Dense(4, kernel_initializer='he_uniform')(x)
    outputs = Softmax()(x)

    model = Model(inputs=inputs, outputs=outputs)

    # Compile model
    opt = SGD(learning_rate=0.001, momentum=0.9)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

    return model

# Create the initial model and save its weights
initial_model = define_cifar10_model()
initial_weights = initial_model.get_weights()

# Function to create a new model with the same initial weights
def create_model_with_initial_weights():
    model = define_cifar10_model()
    model.set_weights(initial_weights)
    return model


# Run the test harness for evaluating the Pre-Trained model
def run_cifar10_test_harness():
    # Load CIFAR-10 dataset
    trainX, trainY, testX, testY = load_dataset()
    trainX, testX = prep_pixels(trainX, testX)
    # Define model
    model = create_model_with_initial_weights()
    # Fit model
    history = model.fit(trainX, trainY, epochs=50, batch_size=64, validation_data=(testX, testY), verbose=1)
    # Save model
    _, acc = model.evaluate(testX, testY, verbose=0)
    print('> %.3f' % (acc * 100.0))
    # learning curves
    model.save('cifar10.keras')

# Entry point, run the CIFAR-10 test harness
run_cifar10_test_harness()

Model for extracting feature vectors from Penultimate layer

In [ ]:
original_model = load_model('cifar10.keras')
original_model.summary()
layer_before_softmax_output = original_model.layers[-2].output
layer_before_softmax_model = Model(inputs=original_model.input, outputs=layer_before_softmax_output)
layer_before_softmax_model.summary()

Dataset Subset selector and feature extractor



In [5]:
#This function takes number of data samples for each class as arguments and returns dataset with same number of datasamples
def subset_cifar10(args):
    if len(args) != 4:
        raise ValueError("Exactly 4 integer arguments are required for the first four classes.")

    # Load CIFAR-10 dataset
    (x_train, y_train), _ = cifar10.load_data()

    # Convert y_train to a 1D array
    y_train = y_train.squeeze()

    # Count occurrences of each class
    class_counts = defaultdict(int)
    for label in y_train:
        class_counts[label] += 1

    # Create a dictionary to store indices of samples for each class
    class_indices = defaultdict(list)
    for i, label in enumerate(y_train):
        class_indices[label].append(i)

    # Select the subset of samples for the first four classes
    subset_indices = []
    for label in range(4):
        num_samples = args[label]
        if num_samples > class_counts[label]:
            raise ValueError(f"Requested {num_samples} samples for class {label}, but there are only {class_counts[label]} samples available.")
        subset_indices.extend(class_indices[label][:num_samples])

    # Shuffle the indices to mix classes
    np.random.shuffle(subset_indices)

    # Extract the subset of images and labels
    x_subset = x_train[subset_indices]
    y_subset = y_train[subset_indices]

    return x_subset, y_subset

def feature_extractor(images,model = original_model):
    images,_ = prep_pixels(images,images)

    feature_vectors = layer_before_softmax_model.predict(images, verbose=0).reshape(len(images), -1)

    row_norms = np.linalg.norm(feature_vectors , axis=1, keepdims=True)

    normalized_feature_vectors = feature_vectors / row_norms

    return normalized_feature_vectors


**BASELINE-1** Under sampling at each client

In [6]:
# Under-sampling for baseline

def count_zeros(number):
    if number <= 0 or number >= 1:
        raise ValueError("Number should be between 0 and 1, exclusive.")

    count = 0
    while number < 1:
        number *= 10
        count += 1
        if number >= 1:
            break

    return count - 1



def base_under_sampling(images, labels, LD):
  LI = min(LD)/max(LD)
  if(LI >= 0.05):
    print("Client aleady Balanced")
    return  images, labels, LD
  x_train = images
  y_train = labels
  distr = LD
  while(LI < 0.05):
    selected_class = np.argmax(distr)
    num = count_zeros(LI)
    class_indices = np.where(y_train == selected_class)[0]
    mc_images = x_train[class_indices]
    feature_vectors = feature_extractor(mc_images)
    cosine_sim_matrix = cosine_similarity(feature_vectors)
    n = cosine_sim_matrix.shape[0]
    #print(n)

    # Compute the mean and variance for each row
    row_means = np.mean(cosine_sim_matrix, axis=1)
    row_variances = np.var(cosine_sim_matrix, axis=1)

    # Sort the rows based on mean values in descending order
    sorted_indices = np.argsort(row_means)[::-1]
    #print(num+1)
    # Select the top 1/10 of the rows with the highest mean values
    selected_indices = sorted_indices[:int((10**(num+1))/5)]

    # Calculate the variance of the selected rows
    selected_var = np.var(cosine_sim_matrix[selected_indices], axis=0)

    # Iterate through the remaining rows and select additional rows one by one
    for i in sorted_indices[int((10**(num+1))/5):]:
        temp_indices = np.append(selected_indices, i)
        temp_var = np.var(cosine_sim_matrix[temp_indices], axis=0)
        # If variance is lower, add the row to the selection
        if np.sum(temp_var) < np.sum(selected_var):
            selected_var = temp_var
            selected_indices = temp_indices
        # Break if the desired number of rows is reached
        if len(selected_indices) == int((10**(num+1))/5):
            break
    #print("images to be removed: ",len(selected_indices)," of class ",(selected_class))
    # Convert lists to numpy arrays for easy indexing


    # Get indices of the samples belonging to the selected class
    selected_class_indices = np.where(y_train == selected_class)[0]

    # Translate samples_to_remove indices to indices in the original dataset
    remove_indices = selected_class_indices[selected_indices]

    # Create masks for removing samples
    mask = np.ones(len(y_train), dtype=bool)
    mask[remove_indices] = False
    # Apply masks to x_train and y_train to remove selected samples
    x_train = x_train[mask]
    y_train = y_train[mask]
    new_distr = np.copy(distr)
    new_distr[selected_class] -= len(remove_indices)
    distr = new_distr
    LI = min(distr)/max(distr)
    #print(LI)
  return x_train, y_train, distr

**Baseline -2** Over Sampling at each client

In [7]:
def img_augmentation(images, save_dir, num_augmented_images):
  prefix='test',
  data_augment = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
  )

  if not os.path.exists(save_dir):
    os.makedirs(save_dir)

  # Convert images to numpy arrays and stack them
  image_arrays = [img_to_array(image) for image in images]
  image_array_batch = np.stack(image_arrays)

  # Reshape the batch array to add an extra dimension
  image_array_batch = image_array_batch.reshape((len(images),) + image_array_batch.shape[1:])

  total_images_to_generate = num_augmented_images
  i = 0

  for batch in data_augment.flow(image_array_batch, batch_size=len(images), save_to_dir=save_dir, save_prefix=prefix, save_format='jpg'):
    i += len(batch)
    if i >= total_images_to_generate:
      break
  return 0


def over_sampling_from_folder_clien(images,labels, LD):
  LI = min(LD)/max(LD)
  #print(LI)
  if(LI >= 0.05):
    print("CLient Already Balanced")
    return images,labels, LD
  new_images = []
  new_labels = []
  image_folder = '/content/generated_images3'
  existing_images = images
  existing_labels = labels
  distr = LD
  while(LI < 0.05):
    selected_class = np.argmin(distr)
    class_indices = np.where(existing_labels == selected_class)[0]
    selected_images = existing_images[class_indices]
    multiple = int(1/(LI))
    img_augmentation(selected_images,image_folder ,multiple)
    # Count the number of existing images for the specified class label
    num_existing_images = len(class_indices)
    #print(num_existing_images, end = " ")
    # Calculate the number of images to add
    num_images_to_add = num_existing_images
    # Collect all filenames in the folder
    filenames = os.listdir(image_folder)
    # Shuffle the filenames
    random.shuffle(filenames)

    # Iterate through the images in the folder
    for filename in filenames:
      filepath = os.path.join(image_folder, filename)
      image = cv2.imread(filepath)
      new_images.append(image)
      new_labels.append(selected_class)
    # Convert lists to numpy arrays
    np_new_images = np.array(new_images)
    np_new_labels = np.array(new_labels)
    # Concatenate new images and labels with existing dataset
    existing_images = np.concatenate((existing_images, np_new_images), axis=0)
    existing_labels = np.concatenate((existing_labels, np_new_labels), axis=0)
    # Update distribution
    new_distribution = np.copy(distr)
    new_distribution[selected_class] += len(new_images)
    print(num_images_to_add," : images added successfully in class",selected_class)
    distr = new_distribution
    LI = min(distr)/max(distr)
    #print(LI)
    for file in glob.glob("/content/generated_images3/*"):
      os.remove(file)
  return  existing_images, existing_labels, distr

**MAIN CODE**

---


All methods defined are for FLICKER


In [8]:

class Server:
    _instance = None  # Class variable to store the single instance

    def __new__(cls, *args, **kwargs):
        if not cls._instance:
            cls._instance = super().__new__(cls, *args, **kwargs)
            # Generate CKKS keys upon instantiation
            cls._instance.__context = ts.context(
                ts.SCHEME_TYPE.CKKS,
                poly_modulus_degree=8192,
                coeff_mod_bit_sizes=[60, 40, 40, 60]
            )
            cls._instance.__context.generate_galois_keys()
            cls._instance.__context.global_scale = 2**40
            cls._instance.__public_key = cls._instance.__context.public_key()
            cls._instance.__relin_keys = cls._instance.__context.relin_keys()
            cls._instance.__total_distr =  None
            cls._instance.enc_norm_distr =  None
            cls._instance.clients = []
        return cls._instance



    def encrypt_with_public_key(self, data):
      return ts.ckks_vector(self.__context, data)

    def restore_with_public_key(self, data):
      return ts.ckks_vector_from(self.__context, data)


    def decryption(self):
      return Client.cumulative_encrypted_distribution.decrypt()


    def decrypt(self, encrypted_data):
      return encrypted_data.decrypt()


    def global_imbalance(self):
      total_distr = [round(x) for x in self.__total_distr]
      return min(total_distr)/max(total_distr)


    def norm_enc(self):
      total_distr = [round(x) for x in self.__total_distr]
      total_distribution = np.linalg.norm(total_distr)
      normalized_distr = [x/total_distribution for x in total_distr]
      return ts.ckks_vector(self.__context, normalized_distr)


    def add_client(self, client):
      self.clients.append(client)


    def cos_sim(self):
      data = []
      client_similarities = np.array(data)
      self.enc_norm_distr = self.norm_enc()
      #print(self.enc_norm_distr.decrypt())
      for client in self.clients:
        temp = client.cos_sim_calc(self.enc_norm_distr)
        temp_decrypt = temp.decrypt()
        client_similarities = np.append(client_similarities, temp_decrypt)
      return client_similarities



    def similarity_comparison(self, enc_selected_vectors, calling_client):
        results = []
        for client in self.clients:
            if client != calling_client:
                result = client.plain_enc_mul(enc_selected_vectors)
                results.append(result)
        return results[0], results[1], results[2]


    def balance_check(self):

      self.__total_distr = self.decryption()
      print("Initial Distr : ", self.__total_distr)
      global_imbalance = self.global_imbalance()
      print("Initial Imbal : ",  global_imbalance)
      global_similarity_cl = self.cos_sim()
      print("initial_global_similarity : ",  global_similarity_cl)
      k = 1
      GI_flag1, GI_flag2 = 0, 0
      sorted_indices = np.argsort(global_similarity_cl)
      round = 0
      while(global_imbalance < 0.1):
        selected_client_index = sorted_indices[-k]
        selected_client = self.clients[selected_client_index]
        print("selected_client : ",selected_client_index)
        enc_results = selected_client.trigger(GI_flag1,GI_flag2, 0)
        if(enc_results == 0):
          k = k + 1
          if(k==5):
            k = 1
            sorted_indices = np.argsort(global_similarity_cl)
            selected_client_index = sorted_indices[-k]
            selected_client = self.clients[selected_client_index]
            if(selected_client.trigger(GI_flag1,GI_flag2, 0) == 0):
              print("That's the best balance you can get")
              print("number of rounds  : ", round)
              break
          GI_flag1, GI_flag2 = 0, 0
          continue
        prev_global_imbalance = global_imbalance
        self.__total_distr = enc_results.decrypt()
        global_imbalance = self.global_imbalance()
        global_similarity_cl = self.cos_sim()
        print("global_imbalance : ", global_imbalance)
        print("global_similarities : ", global_similarity_cl)
        if(prev_global_imbalance >= global_imbalance):
          GI_flag1 = 1
        enc_results = selected_client.trigger(GI_flag1,GI_flag2, 1)
        if(enc_results == 0):
          k = k + 1
          if(k==5):
            k = 1
            sorted_indices = np.argsort(global_similarity_cl)
            selected_client_index = sorted_indices[-k]
            selected_client = self.clients[selected_client_index]
            if(selected_client.trigger(GI_flag1,GI_flag2, 0) == 0):
              print("That's the best balance you can get")
              print("number of rounds  : ", round)
              break
          GI_flag1, GI_flag2 = 0, 0
          continue

        prev_global_imbalance = global_imbalance
        self.__total_distr = enc_results.decrypt()
        global_imbalance = self.global_imbalance()
        global_similarity_cl = self.cos_sim()
        print("global_imbalance : ", global_imbalance)
        print("global_similarities : ", global_similarity_cl)
        if(prev_global_imbalance >= global_imbalance):
          GI_flag2 = 1
        round = round + 1
      print(" Balancing Done in rounds = ", round)
      return 0

In [9]:

import math
import random
class Client:
    cumulative_encrypted_distribution = None  # Class variable to store cumulative encrypted distribution
    def __init__(self, distr, server, client_id):

        if len(distr) != 4:
            raise ValueError("class_samples must be a list or array of four integers.")

        self.context = ts.context(
          ts.SCHEME_TYPE.CKKS,
          poly_modulus_degree=8192,
          coeff_mod_bit_sizes=[60, 40, 40, 60]
        )
        self.context.generate_galois_keys()
        self.context.global_scale = 2**40


        self.__distr = distr  # Private attribute
        self.__dataset = self.__allocate_dataset(distr)  # Allocate and store the dataset
        self.__feature_vectors = feature_extractor(self.__dataset[0])
        self.__imbalance = min(self.__distr)/max(self.__distr)
        self.client_id = client_id
        self.server = server
        self._method_called = False
        # Initialize the variable that will retain its value between calls
        self.i = None
        self.j = None
        self.server.add_client(self)
        self.__update_cumulative_distribution()

    def __allocate_dataset(self, distr):
        return subset_cifar10(distr)

    def cos_sim_calc(self,enc_distr):
      total =  np.linalg.norm(self.__distr)
      norm_distr = [x/total for x in self.__distr]
      #print(norm_distr)
      vec = enc_distr.mul(norm_distr)
      return vec.sum()

    def encrypt_server_pub_key(self):
      total_distribution = sum(self.__distr)
      normalized_distr = [x/total_distribution for x in self.__distr]
      return self.server.encrypt_with_public_key(self.__distr), self.server.encrypt_with_public_key(normalized_distr)

    def use_dataset(self):
      x_subset, y_subset = self.__dataset
      # Example: Print the shape of the dataset
      print("Dataset Shape:", x_subset.shape, y_subset.shape)
      # Example: Return some information about the dataset
      print("feature vector:",self.__feature_vectors.shape)
      return x_subset, y_subset, self.__distr


    def __update_cumulative_distribution(self):
      """
      Updates the cumulative encrypted distribution of all clients with the current client's distribution.
      """
      if Client.cumulative_encrypted_distribution is None:
          encrypted_distribution, _ = self.encrypt_server_pub_key()
          Client.cumulative_encrypted_distribution = encrypted_distribution
      else:
          # Depiction of Second Layer of Encryption for the Cumulative distr in Transit
          serialized_data = Client.cumulative_encrypted_distribution.serialize()          # Serialize the encrypted vector for further encryption
          decimal_representation = list(serialized_data)                                  # Convert to a list of decimal byte values
          encrypted_vector1 = ts.ckks_vector(self.context, decimal_representation)        # Encrpt with client's public Key
          a = encrypted_vector1.decrypt(self.context.secret_key())                        # Decrypt with Client's secret key
          a = [round(i) for i in a]                                                       # Round of to th nearset integer to overcome encryption errors
          serialized_data_from_decimal = bytes(a)
          encrypted_vector = self.server.restore_with_public_key(serialized_data_from_decimal)  # Converting back to the single encryption format
          # Single encryption operations for the data at a client
          Client.cumulative_encrypted_distribution = encrypted_vector
          Client.cumulative_encrypted_distribution = Client.cumulative_encrypted_distribution.add_(self.__distr)

    def plain_enc_mul(self, enc_vector):
      # Perform element-wise multiplication with the client's encrypted vector
      plain_matrix = np.array(self.__feature_vectors)
      plain_matrix = np.transpose(plain_matrix)
      results = []
      for i in range(math.ceil(plain_matrix.shape[1]/3000)):
        result =  enc_vector.matmul(plain_matrix[:,(i * 3000):min(((i+1)*3000),plain_matrix.shape[1])])
        results.append(result)
      return results



    def under_sampling(self, selected_class, feature_vectors):

      cosine_sim_matrix = cosine_similarity(feature_vectors)
      n = cosine_sim_matrix.shape[0]
      #print(n)

      # Compute the mean and variance for each row
      row_means = np.mean(cosine_sim_matrix, axis=1)
      row_variances = np.var(cosine_sim_matrix, axis=1)

      # Sort the rows based on mean values in descending order
      sorted_indices = np.argsort(row_means)[::-1]

      # Select the top 1/10 of the rows with the highest mean values
      selected_indices = sorted_indices[:int(n/5)]

      # Calculate the variance of the selected rows
      selected_var = np.var(cosine_sim_matrix[selected_indices], axis=0)

      # Iterate through the remaining rows and select additional rows one by one
      for i in sorted_indices[int(n/5):]:
          temp_indices = np.append(selected_indices, i)
          temp_var = np.var(cosine_sim_matrix[temp_indices], axis=0)
          # If variance is lower, add the row to the selection
          if np.sum(temp_var) < np.sum(selected_var):
              selected_var = temp_var
              selected_indices = temp_indices
          # Break if the desired number of rows is reached
          if len(selected_indices) == int(n/5):
              break
      print("images selected : ",len(selected_indices))
      samples_to_remove = []
      # Return the indices of the selected rows
      data =[]
      for i in selected_indices:
        similarities = np.array(data)
        enc_sim_1, enc_sim_2, enc_sim_3 = server.similarity_comparison(ts.ckks_vector(self.context,feature_vectors[i]),self)
        for j in range(len(enc_sim_1)):
          temp = enc_sim_1[j].decrypt()
          similarities = np.concatenate((similarities,temp))
        for j in range(len(enc_sim_2)):
          temp = enc_sim_2[j].decrypt()
          similarities = np.concatenate((similarities,temp))
        for j in range(len(enc_sim_3)):
          temp = enc_sim_3[j].decrypt()
          similarities = np.concatenate((similarities,temp))
        c = np.sum(similarities > 0.98)
        if(c >= 300):
          samples_to_remove.append(i)
      print("images to be removed : ",len(samples_to_remove)," of class ",(selected_class))
      # Convert lists to numpy arrays for easy indexing
      x_train = np.array(self.__dataset[0])
      y_train = np.array(self.__dataset[1])
      f_vectors = np.array(self.__feature_vectors)

      # Get indices of the samples belonging to the selected class
      selected_class_indices = np.where(y_train == selected_class)[0]

      # Translate samples_to_remove indices to indices in the original dataset
      remove_indices = selected_class_indices[samples_to_remove]

      # Create masks for removing samples
      mask = np.ones(len(y_train), dtype=bool)
      mask[remove_indices] = False

      # Apply masks to x_train and y_train to remove selected samples
      x_train = x_train[mask]
      y_train = y_train[mask]
      f_vectors = f_vectors[mask]
      Client.cumulative_encrypted_distribution = Client.cumulative_encrypted_distribution.sub_(self.__distr)
      # Update the class distribution
      new_distr = np.copy(self.__distr)
      new_distr[selected_class] -= len(remove_indices)
      self.__distr = new_distr
      my_list = list(self.__dataset)
      my_list[0] = x_train
      my_list[1] = y_train
      self.__dataset = tuple(my_list)
      self.__feature_vectors = f_vectors
      Client.cumulative_encrypted_distribution = Client.cumulative_encrypted_distribution.add_(self.__distr)
      return 0


    def image_augmentation(self, images, save_dir, num_augmented_images):
      prefix='test',
      data_augment = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
      )

      if not os.path.exists(save_dir):
        os.makedirs(save_dir)

      # Convert images to numpy arrays and stack them
      image_arrays = [img_to_array(image) for image in images]
      image_array_batch = np.stack(image_arrays)

      # Reshape the batch array to add an extra dimension
      image_array_batch = image_array_batch.reshape((len(images),) + image_array_batch.shape[1:])

      total_images_to_generate = num_augmented_images
      i = 0

      for batch in data_augment.flow(image_array_batch, batch_size=len(images), save_to_dir=save_dir, save_prefix=prefix, save_format='jpg'):
        i += len(batch)
        if i >= total_images_to_generate:
          break
      return 0


    def over_sampling_from_folder(self, class_label, multiple):
      new_images = []
      new_labels = []
      image_folder = '/content/generated_images3'
      existing_images = self.__dataset[0]
      existing_labels = self.__dataset[1]
      f_vectors = np.array(self.__feature_vectors)
      # Get the indices of the existing images belonging to the specified class label
      class_indices = np.where(existing_labels == class_label)[0]
      images = existing_images[class_indices]
      self.image_augmentation(images,image_folder ,multiple)
      # Count the number of existing images for the specified class label
      num_existing_images = len(class_indices)
      print(num_existing_images, end = " ")
      # Calculate the number of images to add
      num_images_to_add = multiple
      # Collect all filenames in the folder
      filenames = os.listdir(image_folder)
      # Shuffle the filenames
      random.shuffle(filenames)

      # Iterate through the images in the folder
      for filename in filenames:
          filepath = os.path.join(image_folder, filename)
          image = cv2.imread(filepath)
          new_images.append(image)
          new_labels.append(class_label)
      # Convert lists to numpy arrays
      new_images = np.array(new_images)
      new_labels = np.array(new_labels)
      new_feat_vec = feature_extractor(new_images)
      Client.cumulative_encrypted_distribution = Client.cumulative_encrypted_distribution.sub_(self.__distr)
      # Concatenate new images and labels with existing dataset
      combined_images = np.concatenate((existing_images, new_images), axis=0)
      combined_labels = np.concatenate((existing_labels, new_labels), axis=0)
      combined_feat_vec = np.concatenate((f_vectors, new_feat_vec), axis=0)
      # Update distribution
      new_distribution = np.copy(self.__distr)
      new_distribution[class_label] += len(new_images)
      print(num_images_to_add," : images added successfully in class",class_label)

      self.__distr = new_distribution
      my_list = list(self.__dataset)
      my_list[0] = combined_images
      my_list[1] = combined_labels
      self.__dataset = tuple(my_list)
      self.__feature_vectors = combined_feat_vec
      Client.cumulative_encrypted_distribution = Client.cumulative_encrypted_distribution.add_(self.__distr)
      for file in glob.glob("/content/generated_images3/*"): os.remove(file)
      return 0


    def trigger(self, GI_flag1, GI_flag2, sampling):
      self.__imbalance = min(self.__distr)/max(self.__distr)
      print(self.__imbalance)
      if(self.__imbalance >= 0.05):
        print("Client ",self.client_id," is already balanced")
        return 0

      if not self._method_called:
        # If this is the first call, initialize the variable to 0
        self.i = 1
        self.j = 0
        self._method_called = True


      if(sampling == 0):
        if(GI_flag1 == 1):
          self.i += 1
          if(self.i == 3):
            self.i = 0
        sorted_indices = np.argsort(self.__distr)
        # Get index of the smallest value (first element in sorted order)
        selected_class = sorted_indices[-self.i]
        selected_class_indices = np.where(self.__dataset[1] == selected_class)[0]
        #print(len(selected_class_indices))
        selected_feature_vectors = [self.__feature_vectors[index] for index in selected_class_indices]
        self.under_sampling(selected_class, selected_feature_vectors)
        return Client.cumulative_encrypted_distribution

      if(sampling == 1):
        if(GI_flag2 == 1):
          self.j += 1
        sorted_indices = np.argsort(self.__distr)
        # Get index of the smallest value (first element in sorted order)
        selected_class = sorted_indices[0]
        multiple = int(1/(4 * self.__imbalance))
        self.over_sampling_from_folder(selected_class,multiple)
        return Client.cumulative_encrypted_distribution

Distribution

In [ ]:
# Example usage:
server = Server()

"""distr1 = [10, 500, 700, 4000]
distr2 = [20, 700, 500, 3000]
distr3 = [30, 400, 600, 3000]
distr4 = [100, 50, 200, 10]"""

"""distr1 = [10, 30, 700, 4000]
distr2 = [20, 40, 500, 3000]
distr3 = [30, 40, 600, 3000]
distr4 = [50, 50, 200, 10]"""


"""distr1 = [10, 30, 3600, 4000]
distr2 = [20, 40, 2700, 3000]
distr3 = [30, 40, 2600, 3000]
distr4 = [100, 50, 200, 10]"""

distr1 = [2, 100, 600, 4000]
distr2 = [3, 200, 700, 3000]
distr3 = [5, 150, 800, 3000]
distr4 = [30, 50, 20, 10]

client_1 = Client(distr1, server, 0)
client_2 = Client(distr2, server, 1)
client_3 = Client(distr3, server, 2)
client_4 = Client(distr4, server, 3)

Imblance not addressed at all

In [ ]:
x_train = []
y_train = []
X = 0
x_t, y_t, a = client_1.use_dataset()
X+= len(y_t)
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, b = client_2.use_dataset()
X+= len(y_t)
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, c = client_3.use_dataset()
X+= len(y_t)
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, d = client_4.use_dataset()
X+= len(y_t)
x_train.append(x_t)
y_train.append(y_t)
print(a)
print(b)
print(c)
print(d)
print(X)

In [ ]:
# Function to update the client's local model
def client_update(local_model, img, label):
    # Train the local model on client data
    local_model.fit(img, label, epochs=4, batch_size = 64, verbose=0)  # You can adjust the number of epochs
    # Return the updated local model
    return local_model

# Function to update the global model on the server
def server_update(local_models):
    local_weights = [model.get_weights() for model in local_models]
    # Average the weights from all local models
    averaged_weights = [sum(weights) / len(local_models) for weights in zip(*local_weights)]
    # Update the global model with the averaged weights
    updated_global_model = define_cifar10_model()
    updated_global_model.set_weights(averaged_weights)
    return updated_global_model

# Function to evaluate the global model
def evaluate(global_model, img, label):

    _, accuracy = global_model.evaluate(img, label, verbose=0)

    return accuracy

#for plot
rounds = []
accuracies = []

# Load the datasets
_, _, x_test, y_test = load_dataset()
x_train_A, x_train_B = prep_pixels(x_train[0], x_train[1])
x_train_C, x_train_D = prep_pixels(x_train[2], x_train[3])
x_test, x_test = prep_pixels(x_test,x_test)


# Initialize the models for clients 0-4 and 5-9
initial_model_A = create_model_with_initial_weights()
initial_model_B = create_model_with_initial_weights()
initial_model_C = create_model_with_initial_weights()
initial_model_D = create_model_with_initial_weights()
global_model = create_model_with_initial_weights()


y_train_A = to_categorical(y_train[0])
y_train_B = to_categorical(y_train[1])
y_train_C = to_categorical(y_train[2])
y_train_D = to_categorical(y_train[3])
#y_test = to_categorical(y_test)
global_model.summary()
# federated learning
num_rounds = 20

for round_num in range(num_rounds):
    accuracy = evaluate(global_model,x_test, y_test)
    print(f"Round {round_num}: Accuracy = {accuracy * 100}")

    global_weights = global_model.get_weights()
    initial_model_A.set_weights(global_weights)
    initial_model_B.set_weights(global_weights)
    initial_model_C.set_weights(global_weights)
    initial_model_D.set_weights(global_weights)
    initial_model_A = client_update(initial_model_A, x_train_A, y_train_A)
    initial_model_B = client_update(initial_model_B, x_train_B, y_train_B)
    initial_model_C = client_update(initial_model_C, x_train_C, y_train_C)
    initial_model_D = client_update(initial_model_D, x_train_D, y_train_D)
    # Aggregate model updates on the server
    global_model = server_update([initial_model_A, initial_model_B, initial_model_C, initial_model_D])
    rounds.append(round_num)  # Add the current round number
    accuracies.append(accuracy * 100)  # Add the accuracy for the current round

accuracy = evaluate(global_model,x_test, y_test)
print(f"Round {round_num +1}: Accuracy = {accuracy * 100}")
# Evaluate the global model
probabilities = global_model.predict(x_test)
predicted_labels = np.argmax(probabilities, axis=1)
predicted_labels_onehot = to_categorical(predicted_labels)
# Generate classification report
report = classification_report(y_test, predicted_labels_onehot)
print(report)



rounds.append(round_num + 1)  # Add the current round number
accuracies.append(accuracy * 100)  # Add the accuracy for the current round


plt.figure(figsize=(10, 5))
plt.plot(rounds, accuracies, marker='o', label='Federated Model ')
plt.title("Round vs Accuracy")
plt.xlabel("Round Number")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.legend()  # Add a legend to differentiate the lines
plt.ylim(0, 100)
plt.show()

np_accuracy = np.array(accuracies)
np.savetxt('accuracy.txt', np_accuracy, fmt='%f', delimiter=',')
global_model.save('no_bal.keras')


In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize

# Assuming global_model is your trained Keras model and x_test and y_test are your test data

# Predict probabilities
y_pred_prob = global_model.predict(x_test)

# Assuming y_test is one-hot encoded
n_classes = y_test.shape[1]
y_test_binary = y_test

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']  # Adjust the number of colors based on the number of classes
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multiclass')
plt.legend(loc="lower right")
plt.show()

# Print AUC for each class
for i in range(n_classes):
    print(f'AUC for class {i}: {roc_auc[i]}')


Base-Line-1

In [ ]:
x_train[0], y_train[0], LD_0 = base_under_sampling(x_train[0], y_train[0], distr1)
x_train[1], y_train[1], LD_1 = base_under_sampling(x_train[1], y_train[1], distr2)
x_train[2], y_train[2], LD_2 = base_under_sampling(x_train[2], y_train[2], distr3)
x_train[3], y_train[3], LD_3 = base_under_sampling(x_train[3], y_train[3], distr4)
y = LD_0 + LD_1 + LD_2 + LD_3
Y = np.sum(y)

In [ ]:
print(LD_0)
print(LD_1)
print(LD_2)
print(LD_3)
print(Y)
print()
print(distr1)
print(distr2)
print(distr3)
print(distr4)

In [ ]:
# Function to update the client's local model
def client_update(local_model, img, label):
    # Train the local model on client data
    local_model.fit(img, label, epochs=4, batch_size = 64, verbose=0)  # You can adjust the number of epochs
    # Return the updated local model
    return local_model

# Function to update the global model on the server
def server_update(local_models):
    local_weights = [model.get_weights() for model in local_models]
    # Average the weights from all local models
    averaged_weights = [sum(weights) / len(local_models) for weights in zip(*local_weights)]
    # Update the global model with the averaged weights
    updated_global_model = define_cifar10_model()
    updated_global_model.set_weights(averaged_weights)
    return updated_global_model

# Function to evaluate the global model
def evaluate(global_model, img, label):

    _, accuracy = global_model.evaluate(img, label, verbose=0)

    return accuracy

#for plot
rounds1 = []
accuracies1 = []

# Load the datasets
_, _, x_test, y_test = load_dataset()
x_train_A, x_train_B = prep_pixels(x_train[0], x_train[1])
x_train_C, x_train_D = prep_pixels(x_train[2], x_train[3])
x_test, _ = prep_pixels(x_test,_)


# Initialize the models for clients 0-4 and 5-9
initial_model_A = create_model_with_initial_weights()
initial_model_B = create_model_with_initial_weights()
initial_model_C = create_model_with_initial_weights()
initial_model_D = create_model_with_initial_weights()
global_model = create_model_with_initial_weights()

y_train_A = to_categorical(y_train[0])
y_train_B = to_categorical(y_train[1])
y_train_C = to_categorical(y_train[2])
y_train_D = to_categorical(y_train[3])
#y_test = to_categorical(y_test)
global_model.summary()
# federated learning
num_rounds = 20
denom = Y/X
for round_num in range(num_rounds):
    accuracy = evaluate(global_model,x_test, y_test)

    accuracy = accuracy/max(1,denom)
    print(f"Round {round_num}: Accuracy = {accuracy * 100}")

    global_weights = global_model.get_weights()
    initial_model_A.set_weights(global_weights)
    initial_model_B.set_weights(global_weights)
    initial_model_C.set_weights(global_weights)
    initial_model_D.set_weights(global_weights)
    initial_model_A = client_update(initial_model_A, x_train_A, y_train_A)
    initial_model_B = client_update(initial_model_B, x_train_B, y_train_B)
    initial_model_C = client_update(initial_model_C, x_train_C, y_train_C)
    initial_model_D = client_update(initial_model_D, x_train_D, y_train_D)
    # Aggregate model updates on the server
    global_model = server_update([initial_model_A, initial_model_B, initial_model_C, initial_model_D])
    rounds1.append(round_num)  # Add the current round number
    accuracies1.append(accuracy * 100)  # Add the accuracy for the current round

accuracy = evaluate(global_model,x_test, y_test)
accuracy = accuracy/max(1,denom)
print(f"Round {round_num +1}: Accuracy = {accuracy * 100}")
# Evaluate the global model
probabilities = global_model.predict(x_test)
predicted_labels = np.argmax(probabilities, axis=1)
predicted_labels_onehot = to_categorical(predicted_labels)
# Generate classification report
report = classification_report(y_test, predicted_labels_onehot)
print(report)



rounds1.append(round_num + 1)  # Add the current round number
accuracies1.append(accuracy * 100)  # Add the accuracy for the current round


plt.figure(figsize=(10, 5))
plt.plot(rounds1, accuracies1, marker='o', label='Federated Model')
plt.title("Round vs Accuracy")
plt.xlabel("Round Number")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.legend()  # Add a legend to differentiate the lines
plt.ylim(0, 100)
plt.show()
np_accuracy1 = np.array(accuracies1)
np.savetxt('accuracy1.txt', np_accuracy1, fmt='%f', delimiter=',')
global_model.save('US_bal.keras')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Assuming global_model is your trained Keras model and x_test and y_test are your test data

# Predict probabilities
y_pred_prob = global_model.predict(x_test)

# Assuming y_test is one-hot encoded
n_classes = y_test.shape[1]
y_test_binary = y_test

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']  # Adjust the number of colors based on the number of classes
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multiclass')
plt.legend(loc="lower right")
plt.show()

# Print AUC for each class
for i in range(n_classes):
    print(f'AUC for class {i}: {roc_auc[i]}')


Baseline-2

In [ ]:
x_train = []
y_train = []
x_t, y_t, a = client_1.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, b = client_2.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, c = client_3.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, d = client_4.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
print(a)
print(b)
print(c)
print(d)

In [ ]:
x_train[0], y_train[0], LD_0 = over_sampling_from_folder_clien(x_train[0], y_train[0], distr1)
x_train[1], y_train[1], LD_1 = over_sampling_from_folder_clien(x_train[1], y_train[1], distr2)
x_train[2], y_train[2], LD_2 = over_sampling_from_folder_clien(x_train[2], y_train[2], distr3)
x_train[3], y_train[3], LD_3 = over_sampling_from_folder_clien(x_train[3], y_train[3], distr4)
y = LD_0 + LD_1 + LD_2 + LD_3
Y = np.sum(y)


In [ ]:
print(LD_0)
print(LD_1)
print(LD_2)
print(LD_3)
print(Y)
print()
print(distr1)
print(distr2)
print(distr3)
print(distr4)

In [ ]:
# Function to update the client's local model
def client_update(local_model, img, label):
    # Train the local model on client data
    local_model.fit(img, label, epochs=4, batch_size = 64, verbose=0)  # You can adjust the number of epochs
    # Return the updated local model
    return local_model

# Function to update the global model on the server
def server_update(local_models):
    local_weights = [model.get_weights() for model in local_models]
    # Average the weights from all local models
    averaged_weights = [sum(weights) / len(local_models) for weights in zip(*local_weights)]
    # Update the global model with the averaged weights
    updated_global_model = define_cifar10_model()
    updated_global_model.set_weights(averaged_weights)
    return updated_global_model

# Function to evaluate the global model
def evaluate(global_model, img, label):

    _, accuracy = global_model.evaluate(img, label, verbose=0)

    return accuracy

#for plot
rounds2 = []
accuracies2 = []

# Load the datasets
_, _, x_test, y_test = load_dataset()
x_train_A, x_train_B = prep_pixels(x_train[0], x_train[1])
x_train_C, x_train_D = prep_pixels(x_train[2], x_train[3])
x_test, _ = prep_pixels(x_test,_)


# Initialize the models for clients 0-4 and 5-9
initial_model_A = create_model_with_initial_weights()
initial_model_B = create_model_with_initial_weights()
initial_model_C = create_model_with_initial_weights()
initial_model_D = create_model_with_initial_weights()
global_model = create_model_with_initial_weights()

y_train_A = to_categorical(y_train[0])
y_train_B = to_categorical(y_train[1])
y_train_C = to_categorical(y_train[2])
y_train_D = to_categorical(y_train[3])
#y_test = to_categorical(y_test)
global_model.summary()
# federated learning
num_rounds = 20
deno = Y/X



for round_num in range(num_rounds):
    accuracy = evaluate(global_model,x_test, y_test)
    accuracy = accuracy/max(1,deno)
    print(f"Round {round_num}: Accuracy = {accuracy * 100}")

    global_weights = global_model.get_weights()
    initial_model_A.set_weights(global_weights)
    initial_model_B.set_weights(global_weights)
    initial_model_C.set_weights(global_weights)
    initial_model_D.set_weights(global_weights)
    initial_model_A = client_update(initial_model_A, x_train_A, y_train_A)
    initial_model_B = client_update(initial_model_B, x_train_B, y_train_B)
    initial_model_C = client_update(initial_model_C, x_train_C, y_train_C)
    initial_model_D = client_update(initial_model_D, x_train_D, y_train_D)
    # Aggregate model updates on the server
    global_model = server_update([initial_model_A, initial_model_B, initial_model_C, initial_model_D])
    rounds2.append(round_num)  # Add the current round number
    accuracies2.append(accuracy * 100)  # Add the accuracy for the current round

accuracy = evaluate(global_model,x_test, y_test)
accuracy = accuracy/max(1,deno)
print(f"Round {round_num +1}: Accuracy = {accuracy * 100}")
# Evaluate the global model
probabilities = global_model.predict(x_test)
predicted_labels = np.argmax(probabilities, axis=1)
predicted_labels_onehot = to_categorical(predicted_labels)
# Generate classification report
report = classification_report(y_test, predicted_labels_onehot)
print(report)



rounds2.append(round_num + 1)  # Add the current round number
accuracies2.append(accuracy * 100)  # Add the accuracy for the current round


plt.figure(figsize=(10, 5))
plt.plot(rounds2, accuracies2, marker='o', label='Federated Model')
plt.title("Round vs Accuracy")
plt.xlabel("Round Number")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.legend()  # Add a legend to differentiate the lines
plt.ylim(0, 100)
plt.show()
np_accuracy2 = np.array(accuracies2)
np.savetxt('accuracy2.txt', np_accuracy2, fmt='%f', delimiter=',')
global_model.save('OS_bal.keras')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Assuming global_model is your trained Keras model and x_test and y_test are your test data

# Predict probabilities
y_pred_prob = global_model.predict(x_test)

# Assuming y_test is one-hot encoded
n_classes = y_test.shape[1]
y_test_binary = y_test

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']  # Adjust the number of colors based on the number of classes
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multiclass')
plt.legend(loc="lower right")
plt.show()

# Print AUC for each class
for i in range(n_classes):
    print(f'AUC for class {i}: {roc_auc[i]}')


Recommended Scheme

In [ ]:
x_train = []
y_train = []
x_t, y_t, a = client_1.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, b = client_2.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, c = client_3.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, d = client_4.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
print(a)
print(b)
print(c)
print(d)

In [ ]:
server.balance_check()

In [ ]:
x_train = []
y_train = []
x_t, y_t, a = client_1.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, b = client_2.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, c = client_3.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
x_t, y_t, d = client_4.use_dataset()
x_train.append(x_t)
y_train.append(y_t)
y = a + b + c + d
Y = np.sum(y)

print(a)
print(b)
print(c)
print(d)
print(Y)

In [ ]:
# Function to update the client's local model
def client_update(local_model, img, label):
    # Train the local model on client data
    local_model.fit(img, label, epochs=4, batch_size = 64, verbose=0)  # You can adjust the number of epochs
    # Return the updated local model
    return local_model

# Function to update the global model on the server
def server_update(local_models):
    local_weights = [model.get_weights() for model in local_models]
    # Average the weights from all local models
    averaged_weights = [sum(weights) / len(local_models) for weights in zip(*local_weights)]
    # Update the global model with the averaged weights
    updated_global_model = define_cifar10_model()
    updated_global_model.set_weights(averaged_weights)
    return updated_global_model

# Function to evaluate the global model
def evaluate(global_model, img, label):

    _, accuracy = global_model.evaluate(img, label, verbose=0)

    return accuracy

#for plot
rounds3 = []
accuracies3 = []

# Load the datasets
_, _, x_test, y_test = load_dataset()
x_train_A, x_train_B = prep_pixels(x_train[0], x_train[1])
x_train_C, x_train_D = prep_pixels(x_train[2], x_train[3])
x_test, _ = prep_pixels(x_test,_)


# Initialize the models for clients 0-4 and 5-9
initial_model_A = create_model_with_initial_weights()
initial_model_B = create_model_with_initial_weights()
initial_model_C = create_model_with_initial_weights()
initial_model_D = create_model_with_initial_weights()
global_model = create_model_with_initial_weights()

y_train_A = to_categorical(y_train[0])
y_train_B = to_categorical(y_train[1])
y_train_C = to_categorical(y_train[2])
y_train_D = to_categorical(y_train[3])
#y_test = to_categorical(y_test)
global_model.summary()
# federated learning
num_rounds = 20
denom = Y/X
for round_num in range(num_rounds):
    accuracy = evaluate(global_model,x_test, y_test)
    accuracy = accuracy/max(1,denom)
    print(f"Round {round_num}: Accuracy = {accuracy * 100}")
    global_weights = global_model.get_weights()
    initial_model_A.set_weights(global_weights)
    initial_model_B.set_weights(global_weights)
    initial_model_C.set_weights(global_weights)
    initial_model_D.set_weights(global_weights)
    initial_model_A = client_update(initial_model_A, x_train_A, y_train_A)
    initial_model_B = client_update(initial_model_B, x_train_B, y_train_B)
    initial_model_C = client_update(initial_model_C, x_train_C, y_train_C)
    initial_model_D = client_update(initial_model_D, x_train_D, y_train_D)
    # Aggregate model updates on the server
    global_model = server_update([initial_model_A, initial_model_B, initial_model_C, initial_model_D])
    rounds3.append(round_num)  # Add the current round number
    accuracies3.append(accuracy * 100)  # Add the accuracy for the current round

accuracy = evaluate(global_model,x_test, y_test)
accuracy = accuracy/max(1,denom)
print(f"Round {round_num +1}: Accuracy = {accuracy * 100}")
# Evaluate the global model
probabilities = global_model.predict(x_test)
predicted_labels = np.argmax(probabilities, axis=1)
predicted_labels_onehot = to_categorical(predicted_labels)
# Generate classification report
report = classification_report(y_test, predicted_labels_onehot)
print(report)



rounds3.append(round_num + 1)  # Add the current round number
accuracies3.append(accuracy * 100)  # Add the accuracy for the current round


plt.figure(figsize=(10, 5))
plt.plot(rounds3, accuracies3, marker='o', label='Federated Model')
plt.title("Round vs Accuracy")
plt.xlabel("Round Number")
plt.ylabel("Accuracy (%)")
plt.grid(True)
plt.legend()  # Add a legend to differentiate the lines
plt.ylim(0, 100)
plt.show()

np_accuracy3 = np.array(accuracies3)
np.savetxt('accuracy3.txt', np_accuracy3, fmt='%f', delimiter=',')
global_model.save('proposed_bal.keras')

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
import numpy as np

# Assuming global_model is your trained Keras model and x_test and y_test are your test data

# Predict probabilities
y_pred_prob = global_model.predict(x_test)

# Assuming y_test is one-hot encoded
n_classes = y_test.shape[1]
y_test_binary = y_test

# Calculate ROC curve and AUC for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_binary[:, i], y_pred_prob[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Plot ROC curve for each class
plt.figure()
colors = ['aqua', 'darkorange', 'cornflowerblue', 'green']  # Adjust the number of colors based on the number of classes
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], color=colors[i % len(colors)], lw=2,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Multiclass')
plt.legend(loc="lower right")
plt.show()

# Print AUC for each class
for i in range(n_classes):
    print(f'AUC for class {i}: {roc_auc[i]}')


In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(rounds, accuracies, marker='o', markersize=8, linewidth=1, label='Training with Class Imbalance')
plt.plot(rounds1, accuracies1, marker='h', markersize=8, linewidth=1, label='Under Sampling')
plt.plot(rounds2, accuracies2, marker='s', markersize=8, linewidth=1, label='Over Sampling')
plt.plot(rounds3, accuracies3, marker='>', markersize=8, linewidth=1, label='FLICKER')

plt.title("Round vs Accuracy", fontsize=16, fontweight='bold')
plt.xlabel("Round Number", fontsize=14, fontweight='bold')
plt.ylabel("Normalized Accuracy (%)", fontsize=14, fontweight='bold')
plt.grid(True)
plt.legend(fontsize=14)  # Add a legend to differentiate the lines
plt.ylim(20, 80)
plt.show()